<a href="https://colab.research.google.com/github/soumyadiptapete/GNN_practice/blob/main/mlp_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
from torch.nn import Linear
import torch.nn.functional as F

In [4]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.8 MB/s eta 0:00:00


In [5]:
from torch_geometric.datasets import FacebookPagePage
dataset=FacebookPagePage(root='.')
data=dataset[0]

Processing...
Done!


In [11]:
print(f'is directed: {data.is_directed()}')
print(f'number of nodes {data.x.shape[0]}')
print(f'data has isolated nodes {data.has_isolated_nodes()}')
print(f'data has loop: {data.has_self_loops()}')

is directed: False
number of nodes 22470
data has isolated nodes False
data has loop: True


In [14]:
data.train_mask=range(18000)
data.val_mask=range(18001,20000)
data.test_mask=range(20001,22470)


In [16]:
# make dataset as table with dims Nxd
import pandas as pd
df_x=pd.DataFrame(data.x.numpy())
df_x['label']=pd.DataFrame(data.y)
df_x.head()

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,label
0,-0.262576,-0.276483,-0.262350,-0.299327,-0.299159,-0.270681,-0.307757,-0.269733,-0.25101,-0.308343,...,-0.273229,-0.223700,-0.284379,-0.224216,-0.209509,-0.255755,-0.21514,-0.375903,-0.223836,0
1,-0.262576,-0.276483,-0.262350,-0.299327,-0.299159,-0.270681,-0.307757,-0.269733,-0.25101,-0.308343,...,-0.234818,-0.223700,-0.284379,-0.197935,-0.147256,-0.255755,-0.21514,-0.364134,-0.128634,2
2,-0.262576,-0.265053,-0.262350,-0.299327,-0.299159,-0.270681,-0.307757,-0.210461,-0.25101,3.222161,...,-0.273229,-0.223700,-0.284379,-0.224216,-0.209509,-0.255755,-0.21514,-0.375903,-0.223836,1
3,-0.246378,-0.276483,-0.241991,-0.299327,-0.299159,-0.270681,-0.307051,-0.269733,-0.25101,-0.308343,...,-0.273229,-0.223700,-0.265534,-0.080353,-0.209509,-0.250560,-0.18026,-0.375903,-0.223836,2
4,-0.262576,-0.276483,-0.262350,-0.299327,-0.299159,-0.270681,-0.307757,-0.269733,-0.25101,-0.308343,...,-0.273229,-0.175312,-0.272613,-0.224216,-0.181153,-0.255755,-0.21514,-0.370639,-0.223836,3


In [63]:
import numpy as np
#create MLP class
class MLP(torch.nn.Module):
  def __init__(self,dim_in, dim_h,dim_out):
    super().__init__()
    self.Linear1=Linear(dim_in,dim_h)
    self.Linear2=Linear(dim_h,dim_out)

  def forward(self,x):
    x=self.Linear1(x)
    x=F.relu(x)
    x=self.Linear2(x)
    return F.log_softmax(x,dim=1) #softmax along the 1 dimension whihc has the features for a node


  def fit(self,data,epochs):
    criterion=torch.nn.CrossEntropyLoss()
    optimizer=torch.optim.Adam(self.parameters(), lr=0.01, weight_decay=5e-4)

    y=data.y#labels


    for epoch in range(epochs+1):
      out=self(data.x)#output of network forward pass
      optimizer.zero_grad()
      loss=criterion(out[data.train_mask],y[data.train_mask])
      # print train loss and validation loss
      if epoch%20==0:
        val_loss=criterion(out[data.val_mask],y[data.val_mask])
        print(f'Train Loss : {loss}')
        print(f'Validation loss: {val_loss}')

      loss.backward()
      optimizer.step()



In [64]:
#initialize
mlp=MLP(dataset.num_features, 16,dataset.num_classes)
print(mlp)

MLP(
  (Linear1): Linear(in_features=128, out_features=16, bias=True)
  (Linear2): Linear(in_features=16, out_features=4, bias=True)
)


In [65]:
mlp.fit(data,epochs=100)

Train Loss : 1.420718789100647
Validation loss: 1.4181002378463745
Train Loss : 0.6756566166877747
Validation loss: 0.6867074966430664
Train Loss : 0.5794861912727356
Validation loss: 0.6108224987983704
Train Loss : 0.5511820316314697
Validation loss: 0.597206175327301
Train Loss : 0.5351370573043823
Validation loss: 0.592629075050354
Train Loss : 0.5220404267311096
Validation loss: 0.5960835814476013
